# MINE 4201- Sistemas de Recomendación
# Taller 1 -  Modelos Colaborativos

### **Integrantes**
* Cristian C. Moreno Mojica(c.morenom@uniandes.edu.co) 
* Juan J. Ovalle Villamil(jj.ovalle@uniandes.edu.co) 
* Maria C. Rodríguez Niño (mc.rodriguezn12@uniandes.edu.co)

## Librerias

In [80]:
import pandas as pd 
import numpy as np
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import train_test_split
from surprise import KNNBasic
from surprise import accuracy

## 1) Conocimiento del dataset de trabajo

In [10]:
#Importar base 1. 
user_info = pd.read_csv('data/lastfm-dataset-1K/userid-profile.tsv',sep='\t')

In [13]:
print(user_info.shape)
display(user_info.head(10))
print('La base user_info muestra el genero, la edad, el pais de registro y la fecha de registro de cada usuario.')

(992, 5)


,#id,gender,age,country,registered
0,user_000001,m,NaN,Japan,"Aug 13, 2006"
1,user_000002,f,NaN,Peru,"Feb 24, 2006"
2,user_000003,m,22.0,United States,"Oct 30, 2005"
3,user_000004,f,NaN,NaN,"Apr 26, 2006"
4,user_000005,m,NaN,Bulgaria,"Jun 29, 2006"
5,user_000006,NaN,24.0,Russian Federation,"May 18, 2006"
6,user_000007,f,NaN,United States,"Jan 22, 2006"
7,user_000008,m,23.0,Slovakia,"Sep 28, 2006"
8,user_000009,f,19.0,United States,"Jan 13, 2007"
9,user_000010,m,19.0,Poland,"May 4, 2006"


La base user_info muestra el genero, la edad, el pais de registro y la fecha de registro de cada usuario.


In [15]:
#Importar base 2. 
user_interact = pd.read_csv('data/lastfm-dataset-1K/userid-timestamp-artid-artname-traid-traname.tsv',
                  sep='\t',
                 names = ['UserId','TimeStamp','ArtId','ArtName','TraId','TraName'])

In [16]:
print(user_interact.shape)
display(user_interact.head(10))
print('La base user_interact muestra las interacciones de cada uno de los usuarios con los artistas y las canciones disponibles.')

(19098862, 6)


,UserId,TimeStamp,ArtId,ArtName,TraId,TraName
0,user_000001,2009-05-04T23:08:57Z,f1b1cf71-bd35-4e99-8624-24a6e15f133a,Deep Dish,NaN,Fuck Me Im Famous (Pacha Ibiza)-09-28-2007
1,user_000001,2009-05-04T13:54:10Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Composition 0919 (Live_2009_4_15)
2,user_000001,2009-05-04T13:52:04Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Mc2 (Live_2009_4_15)
3,user_000001,2009-05-04T13:42:52Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Hibari (Live_2009_4_15)
4,user_000001,2009-05-04T13:42:11Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Mc1 (Live_2009_4_15)
5,user_000001,2009-05-04T13:38:31Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,To Stanford (Live_2009_4_15)
6,user_000001,2009-05-04T13:33:28Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Improvisation (Live_2009_4_15)
7,user_000001,2009-05-04T13:23:45Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Glacier (Live_2009_4_15)
8,user_000001,2009-05-04T13:19:22Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Parolibre (Live_2009_4_15)
9,user_000001,2009-05-04T13:13:38Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Bibo No Aozora (Live_2009_4_15)


La base user_interact muestra las interacciones de cada uno de los usuarios con los artistas y las canciones disponibles.


In [29]:
#Numero de artitas diferentes escuchados por cada usuario  y frecuencia 
user_interact.groupby(['UserId', 'ArtId','ArtName']).size().reset_index(name = 'Frecuencia')

,UserId,ArtId,ArtName,Frecuencia
0,user_000001,00c73a38-a449-4990-86ca-5088dde1b8df,Dj Rels,2
1,user_000001,012a77c9-c897-494f-87d0-0a730996494d,Vincent,1
2,user_000001,014ba96b-b8da-49e3-8a2b-b720ae42e84c,Dbx,3
3,user_000001,01ce7548-dab4-4ca6-8dfc-8e2e4b50d461,Lars Bartkuhn,4
4,user_000001,03282c56-8a24-42f4-8bfc-96188933aefa,Angela Bofill,4
...,...,...,...,...
819780,user_001000,ff3e120c-c929-4ad4-bbbd-04d1689ffc54,Gifts From Enola,1
819781,user_001000,ff6e677f-91dd-4986-a174-8db0474b1799,Jack Johnson,42
819782,user_001000,ff9deaae-da4f-42b7-a19e-36fedd3fc706,Ben Kweller,2
819783,user_001000,ffb18e19-64a4-4a65-b4ce-979e00c3c69d,The Album Leaf,137


In [30]:
#Numero de canciones diferentes escuchadas por cada usuario y frecuencia
user_interact.groupby(['UserId', 'TraId','TraName']).size().reset_index(name = 'Frecuencia')

,UserId,TraId,TraName,Frecuencia
0,user_000001,00237585-8e04-4cd5-a785-f2185492ab0b,Monday Nightcap,2
1,user_000001,0024d72c-136f-49f2-9078-ce4b39b94d3f,Something In The Way,4
2,user_000001,0025055f-39c3-43e2-b874-3bf42bbc9212,Squeeze Me (Feat. Ben Westbeech),2
3,user_000001,002e254d-4624-49f4-b78a-b40711b9e4f3,Tak 4,3
4,user_000001,00b07689-ec4c-4773-94ce-06f3d198431e,Wanderlust,4
...,...,...,...,...
3957890,user_001000,ffb47f0b-fa38-4d16-ad44-b3b1b76a93cd,I Want To Be The Boy To Warm Your Mother'S Heart,4
3957891,user_001000,ffbf7562-62e9-42e9-aab1-4d0572a05cf5,The Big U,7
3957892,user_001000,ffc62f6c-7712-4534-8085-88676647947f,Run For Your Life,1
3957893,user_001000,ffc9d3fd-8153-4ccf-b6e9-06d9f5df72b2,The Skin Of My Yellow Country Teeth,5


In [28]:
#Top 10 artistas escuchados
user_interact.groupby(['ArtId','ArtName']).size().reset_index(name = 'Frecuencia').sort_values(by = 'Frecuencia',  ascending = False).head(10)

,ArtId,ArtName,Frecuencia
70252,a74b1b7f-71a5-4011-9441-d0b5e4122711,Radiohead,115099
74262,b10bbbfc-cf9e-42e0-be17-e2c3e1d2600d,The Beatles,100126
77135,b7ffd2af-418f-4be2-bdd1-22f8b48613da,Nine Inch Nails,84317
65734,9c9f1380-2516-4fc9-a3e6-f9f61941d090,Muse,63139
85720,cc197bad-dc9c-440d-a5b5-d52ba2e14234,Coldplay,62212
55945,8538e728-ca0b-4321-b7e5-cff6565dd4c0,Depeche Mode,59609
55359,83d91898-7763-47d7-b03b-b92132375c47,Pink Floyd,58484
99,0039c7ae-e1a7-4a7d-9b49-0cbc716821a6,Death Cab For Cutie,58046
55629,847e8284-8582-4b0e-9c26-b042a4f49e57,Placebo,53467
1610,03ad1736-b7c9-412a-b442-82536d63a5c4,Elliott Smith,50202


In [33]:
#Top 10 canciones escuchadas
user_interact.groupby(['TraId','TraName']).size().reset_index(name = 'Frecuencia').sort_values(by = 'Frecuencia',  ascending = False).head(10)

,TraId,TraName,Frecuencia
821625,db16d0b3-b8ce-4aa8-a11a-e4d53cc7f8a6,Such Great Heights,3991
476769,7f1f45c0-0101-49e9-8d69-23951d271163,Love Will Tear Us Apart,3651
593060,9e2ad5bc-c6f9-40d2-a36f-3122ee2072a3,Karma Police,3533
293118,4e17b118-70a6-4c1f-b326-b4ce91fd3fad,Soul Meets Body,3479
957215,ff1e3e1a-f6e8-4692-b426-355880383bb6,Supermassive Black Hole,3463
822442,db4c9220-df76-4b42-b6f5-8bf52cc80f77,Heartbeats,3155
932176,f874c752-65bc-4d50-ac7e-932243ae9f02,Rebellion (Lies),3047
363124,60e94685-0481-4d3d-bd84-11c389d9b2a5,Starlight,3040
710562,bd782340-6fa5-4b52-aa5a-ceafb9bc0340,Gimme More,3002
280955,4ad08552-6c35-49ed-bcc6-6822c8f9dfd8,When You Were Young,2997


## 2) Preprocesamiento de datos

### a) Transforme los datos correspondientes a la interacción entre usuarios e ítems, implementando una estrategia para convertir estos datos en unos que sean compatibles con los modelos vistos en clase. Justifique en el informe sus decisiones en este paso.

### Artistas

In [105]:
%%time
aux_artistas = user_interact.groupby('UserId').agg({'ArtId':'nunique'}).reset_index().rename(columns = {'ArtId':'Nart'})

Wall time: 10.9 s


In [106]:
# Armamos base ratings de ARTISTAS que contiene frecuencia absoluta, logaritimca base 10 y normalizada.
ratings_artistas = user_interact.groupby(['UserId', 'ArtId']).size().reset_index(name = 'frecuencia')
ratings_artistas['log_frecuencia'] = np.log10(ratings_artistas['frecuencia'])
print(ratings_artistas.shape)
ratings_artistas.head()

(819470, 4)


,UserId,ArtId,frecuencia,log_frecuencia
0,user_000001,00c73a38-a449-4990-86ca-5088dde1b8df,2,0.301030
1,user_000001,012a77c9-c897-494f-87d0-0a730996494d,1,0.000000
2,user_000001,014ba96b-b8da-49e3-8a2b-b720ae42e84c,3,0.477121
3,user_000001,01ce7548-dab4-4ca6-8dfc-8e2e4b50d461,4,0.602060
4,user_000001,03282c56-8a24-42f4-8bfc-96188933aefa,4,0.602060


In [108]:
#Left join con la base auxiliar
ratings_artistas_merged = ratings_artistas.merge(aux_artistas, on = 'UserId', how = 'left')
ratings_artistas_merged['normalizada'] = ratings_artistas_merged['frecuencia']/ratings_artistas_merged['Nart']
ratings_artistas_merged.head()

,UserId,ArtId,frecuencia,log_frecuencia,Nart,normalizada
0,user_000001,00c73a38-a449-4990-86ca-5088dde1b8df,2,0.301030,578,0.00346
1,user_000001,012a77c9-c897-494f-87d0-0a730996494d,1,0.000000,578,0.00173
2,user_000001,014ba96b-b8da-49e3-8a2b-b720ae42e84c,3,0.477121,578,0.00519
3,user_000001,01ce7548-dab4-4ca6-8dfc-8e2e4b50d461,4,0.602060,578,0.00692
4,user_000001,03282c56-8a24-42f4-8bfc-96188933aefa,4,0.602060,578,0.00692


### Canciones

In [119]:
%%time
aux_canciones = user_interact.groupby('UserId').agg({'TraId':'nunique'}).reset_index().rename(columns = {'TraId':'Ntra'})

Wall time: 14.1 s


In [116]:
# Armamos base ratings de ARTISTAS que contiene frecuencia absoluta, logaritimca base 10 y normalizada.
ratings_canciones = user_interact.groupby(['UserId', 'TraId']).size().reset_index(name = 'frecuencia')
ratings_canciones['log_frecuencia'] = np.log10(ratings_canciones['frecuencia'])
print(ratings_canciones.shape)
ratings_canciones.head()

(3957807, 4)


,UserId,TraId,frecuencia,log_frecuencia
0,user_000001,00237585-8e04-4cd5-a785-f2185492ab0b,2,0.301030
1,user_000001,0024d72c-136f-49f2-9078-ce4b39b94d3f,4,0.602060
2,user_000001,0025055f-39c3-43e2-b874-3bf42bbc9212,2,0.301030
3,user_000001,002e254d-4624-49f4-b78a-b40711b9e4f3,3,0.477121
4,user_000001,00b07689-ec4c-4773-94ce-06f3d198431e,4,0.602060


In [121]:
#Left join con la base auxiliar
ratings_canciones_merged = ratings_canciones.merge(aux_canciones, on = 'UserId', how = 'left')
ratings_canciones_merged['normalizada'] = ratings_canciones_merged['frecuencia']/ratings_canciones_merged['Ntra']
ratings_canciones_merged.head()

,UserId,TraId,frecuencia,log_frecuencia,Ntra,normalizada
0,user_000001,00237585-8e04-4cd5-a785-f2185492ab0b,2,0.301030,2325,0.00086
1,user_000001,0024d72c-136f-49f2-9078-ce4b39b94d3f,4,0.602060,2325,0.00172
2,user_000001,0025055f-39c3-43e2-b874-3bf42bbc9212,2,0.301030,2325,0.00086
3,user_000001,002e254d-4624-49f4-b78a-b40711b9e4f3,3,0.477121,2325,0.00129
4,user_000001,00b07689-ec4c-4773-94ce-06f3d198431e,4,0.602060,2325,0.00172


### b) Tome los datos compatibles con modelos colaborativos y pártalos en dos conjuntos: un grupo de datos le sirve para construir el modelo y el resto para medir sus predicciones. Sepárelos en archivos distintos.

In [143]:
ratings_artistas_merged.describe()

,frecuencia,log_frecuencia,Nart,normalizada
count,819470.000000,819470.000000,819470.000000,819470.000000
mean,22.573138,0.625399,1720.889599,0.032702
std,123.272152,0.663310,1399.023137,0.408378
min,1.000000,0.000000,1.000000,0.000128
25%,1.000000,0.000000,698.000000,0.000873
50%,3.000000,0.477121,1319.000000,0.002410
75%,11.000000,1.041393,2223.000000,0.010000
max,26496.000000,4.423180,7809.000000,236.571429


### Artistas - frecuencia

In [111]:
reader = Reader( rating_scale = ( min(ratings_artistas_merged['frecuencia']), max(ratings_artistas_merged['frecuencia']) ) )
#Se crea el dataset a partir del dataframe
surprise_dataset = Dataset.load_from_df( ratings_artistas_merged[ [ 'UserId', 'ArtId', 'frecuencia' ] ], reader )

In [113]:
train_set, test_set = train_test_split(surprise_dataset, test_size=.2)

In [124]:
# se crea un modelo knnbasic item-item con similitud coseno 
sim_options = {'name': 'cosine',
               'user_based': True}

algo = KNNBasic(k = 20, min_k = 2, sim_options = sim_options)

In [125]:
algo.fit(trainset = train_set)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [141]:
test_predictions = algo.test(test_set)

In [142]:
accuracy.rmse( test_predictions, verbose = True )

RMSE: 135.8008


135.8008124662396

### Artistas - log frecuencia

In [147]:
reader = Reader( rating_scale = ( min(ratings_artistas_merged['log_frecuencia']), max(ratings_artistas_merged['log_frecuencia']) ) )
#Se crea el dataset a partir del dataframe
surprise_dataset = Dataset.load_from_df( ratings_artistas_merged[ [ 'UserId', 'ArtId', 'log_frecuencia' ] ], reader )
train_set, test_set = train_test_split(surprise_dataset, test_size=.2)

In [148]:
sim_options = {'name': 'pearson',
               'user_based': True}

algo = KNNBasic(k = 20, min_k = 2, sim_options = sim_options)
algo.fit(trainset = train_set)
test_predictions = algo.test(test_set)
accuracy.rmse( test_predictions, verbose = True )

Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 0.6388


0.638812911375371

### Artistas - jaccard

In [151]:
sim_options = {'name': 'jaccard',
               'user_based': True}

algo = KNNBasic(k = 20, min_k = 2, sim_options = sim_options)
algo.fit(trainset = train_set)


Computing the jaccard similarity matrix...


NameError: Wrong sim name jaccard. Allowed values are cosine, msd, pearson, pearson_baseline.